In [ ]:
#pip install tensorflow tensorflow_datasets

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf
import tensorflow_datasets as tfds
import os


2023-04-14 13:36:30.756438: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
batch_size = 64
num_classes = 10
epochs = 30

In [2]:
def load_data():
    
    def predprocess_image(image,label):
        image = tf.image.convert_image_dtype(image,tf.float32)
        return image, label
    ds_train, info = tfds.load("cifar10",with_info=True, split="train",as_supervised=True)
    ds_test = tfds.load("cifar10",split="test",as_supervised=True)

    ds_train = ds_train.repeat().shuffle(1024).map(predprocess_image).batch(batch_size)

    ds_test = ds_test.repeat().shuffle(1024).map(predprocess_image).batch(batch_size)

    return ds_train, ds_test, info

In [ ]:
def create_model(input_shape):
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(3,3),padding="same", input_shape=input_shape))
    model.add(Activation("relu"))
    model.add(Conv2D(filters=32, kernel_size=(3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())

    model.add(Dense(1024))
    model.add(Activation("relu")) #Sigmoid, Tanh, ReLU
    model.add(Dropout(0.5)) #Регуляция
    model.add(Dense(num_classes,activation="softmax"))

    model.summary()

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model
    
    


In [3]:
if __name__ == "__main__":
    ds_train, ds_test,info=load_data()
    model = create_model(input_shape=info.features["image"].shape)

 

    logdir = os.path.join("logs", "cifar10-model-v1")

    tensorboard = TensorBoard(log_dir=logdir)
    if not os.path.isdir("results"):
        os.mkdir("results")

    model.fit(ds_train, epochs=epochs, validation_data=ds_test, verbose=1, 
              steps_per_epoch=info.splits["train"].num_examples // batch_size,
              validation_steps=info.splits["test"].num_examples // batch_size,
              callbacks = [tensorboard])
    
    model.save("results/cifar10-model-v1.h5")

NameError: name 'tfds' is not defined

In [ ]:


from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
categories = {
    0: "airplane",
    1: "automobile",
    2: "bird",
    3: "cat",
    4: "deer",
    5: "dog",
    6: "frog",
    7: "horse",
    8: "ship",
    9: "truck",
}

In [ ]:
ds_train, ds_test, info = load_data()

model = load_model("results/cifar10-model-v1.h5")

In [ ]:
#loss, accuracy = model.evaluate(, y_test)
#print("Test accuracy:",accuracy*100,"%")


data_sample = next(iter(ds_test))
sample_image = data_sample[0].numpy()[0]
sample_label = categories[data_sample[1].numpy()[0]]
prediction = np.argmax(model.predict(sample_image.reshape(-1, *sample_image.shape))[0])

print("Predicted label:",categories[prediction])
print("True label:", sample_label)

# Display the sample image
plt.imshow(sample_image)
plt.show()
print(sample_image.shape)

In [ ]:
from PIL import Image
import numpy as np

# загрузить изображение
image = Image.open("dog.jpg")

# изменить размер изображения
image = image.resize((32, 32))

# преобразовать изображение в массив numpy
image_array = np.array(image)

# выполнить нормализацию значений пикселей
normalized_image_array = (image_array.astype(np.float32) / 255.0 - 0.5) / 0.5

# добавить измерение пакета к массиву
processed_image = np.expand_dims(normalized_image_array, axis=0)
processed_image = np.squeeze(processed_image)
prediction = np.argmax(model.predict(processed_image.reshape(-1, *processed_image.shape))[0])

# вывести результат
print("Predicted label:", categories[prediction])

plt.imshow(processed_image)
plt.show()
print(processed_image.shape)


In [ ]:
from PIL import Image
import numpy as np

# загрузить изображение
image = Image.open("cat.jpg")

# изменить размер изображения
image = image.resize((32, 32))

# преобразовать изображение в массив numpy
image_array = np.array(image)

# выполнить нормализацию значений пикселей
normalized_image_array = (image_array.astype(np.float32) / 255.0 - 0.5) / 0.5

# добавить измерение пакета к массиву
processed_image = np.expand_dims(normalized_image_array, axis=0)
processed_image = np.squeeze(processed_image)

plt.imshow(processed_image)
plt.show()
print(processed_image.shape)

prediction = np.argmax(model.predict(processed_image.reshape(-1, *processed_image.shape))[0])

# вывести результат
print("Predicted label:", categories[prediction])


# преобразовать изображение в массив numpy
image_array = np.array(image)

# выполнить нормализацию значений пикселей
normalized_image_array = (image_array.astype(np.float32) / 255.0 - 0.5) / 0.5

# добавить измерение пакета к массиву
processed_image = np.expand_dims(normalized_image_array, axis=0)

# выполнить предсказание на обработанном изображении
prediction = model.predict(processed_image)

# вывести вероятности для каждого класса
for i, proba in enumerate(prediction[0]):
    print(f"{categories[i]}: {proba}")




In [ ]:
Существует множество причин, почему обученная нейронная сеть может плохо классифицировать изображения. Ниже перечислены некоторые из наиболее распространенных проблем:
Недостаточность объема данных для обучения. Чтобы обучить нейронную сеть на задачу классификации изображений, нужно иметь большой и разнообразный набор данных. Если количество обучающих примеров недостаточно, то сеть может не научиться распознавать изображения на новых данных.
Недостаточная глубина сети. Если сеть слишком мала, она может не быть способна выделять достаточно сложные признаки изображения, чтобы классифицировать его правильно.
Неправильный выбор архитектуры сети. Различные архитектуры сетей лучше подходят для разных задач классификации изображений. Например, для классификации изображений лиц лучше использовать сверточные нейронные сети, а для классификации изображений цифр лучше использовать полносвязные нейронные сети.
Неправильное предобработка изображений. Изображения должны быть предобработаны перед подачей их на вход сети. Если этот процесс неправильно выполнен, то сеть может получить неправильную информацию о изображении.
Ошибки в процессе обучения. Обучение нейронной сети может быть неправильно настроено или неоптимальным. Например, сеть может быть переобученной на обучающем наборе, что приведет к плохой обобщающей способности на новых данных.
Недостаточное количество эпох обучения. Если обучение сети было недостаточно продолжительным, то сеть может не успеть усвоить достаточное количество информации для классификации изображений.
Неправильно обработаны данные для анализа нейронной сетью, так например в примере нейронная сеть выдавала ошибку при распознании лошади, по ее мнению это был аэроплан. Данная ошибка вознилка потому что изображение было не информативным, а именно не контрасным. Смотря на очетания изображение которое имело разрешение 32 на 32 пикселя, на изображении плохо видно было лошадь, хорошо видно было очернания неба и земли, поэтому нейросеть допустила ошибку




